todo
1. optimize hyperparameters
2. clean dataset
    - balance
3. data augmentation
4. add requirments.txt
5. correct title (of file as well)
6. go through all variable names
7. ask which types of groups should "be in focus". Do disabled people count?
8. more advaned duplicate dropping?
9. balancing the dataset needed?
10. dedicated train test split?
11. data augmentation only on train?
12. remove duplicates a second time
13. train test split after tokenization?

# Hate Detection

In [227]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix, f1_score

from keras.callbacks import EarlyStopping
from keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


from random import sample

### Data Exploration

In [228]:
data = pd.read_csv('hate-text.csv')
data

,case_id,test_case,label_gold,target_ident
0,1,I hate women.,hateful,women
1,2,I hate trans people.,hateful,trans people
2,3,I hate gay people.,hateful,gay people
3,4,I hate black people.,hateful,black people
4,5,I hate disabled people.,hateful,disabled people
...,...,...,...,...
3896,3897,Gay people are all complete 4ssholes.,hateful,gay people
3897,3898,Black people are all complete 4ssholes.,hateful,black people
3898,3899,Disabled people are all complete 4ssholes.,hateful,disabled people
3899,3900,Muslims are all complete 4ssholes.,hateful,Muslims


The dataset contains some missing values in the target_ident column, which we'll have to deal with later

In [229]:
# Using display() to display both at the same time. Won't comment any future use of display() (for obvious reasons)
display(
    data.isnull().any(),
    data[data.isna().any(axis=1)]
)

case_id         False
test_case       False
label_gold      False
target_ident     True
dtype: bool

,case_id,test_case,label_gold,target_ident
1271,1272,"Fucking hell, what a day.",non-hateful,NaN
1272,1273,That's a great fucking idea.,non-hateful,NaN
1273,1274,No fucking way.,non-hateful,NaN
1274,1275,I'm so fucking excited.,non-hateful,NaN
1275,1276,Are you fucking kidding me?,non-hateful,NaN
...,...,...,...,...
3097,3098,Pro lifers should get the fuck out of our coun...,non-hateful,NaN
3098,3099,Most gun dealers are seriously fucked in the h...,non-hateful,NaN
3099,3100,"By nature, corporate accountants are just shit...",non-hateful,NaN
3100,3101,Most real estate agents are absolute bastards.,non-hateful,NaN


No duplicate texts found

In [230]:
print(f'''duplicate texts: {data['test_case'].duplicated().any()}''')

duplicate texts: False


While the dataset is quite balanced in the target groups, it's still assymetric when it comes to the number of hate/non-hate texts

In [231]:
display(
    pd.DataFrame(data['label_gold'].value_counts()),
    pd.DataFrame(data['target_ident'].value_counts())
)

,count
label_gold,
hateful,2659
non-hateful,1242


,count
target_ident,
gay people,577
women,535
disabled people,510
Muslims,510
black people,504
trans people,485
immigrants,485


No unusually short/long texts

In [232]:
print(
    f'Shortest: {len(data["test_case"].iloc[0])}\n'
    f'Longest: {len(data["test_case"].iloc[-1])}'
)

Shortest: 14
Longest: 38


### Data cleaning

Drop and rename columns to something more suitable

In [233]:
data = data.drop('case_id', axis=1)
data = data.rename(
    columns=
    {
        'test_case': 'text',
        'label_gold': 'label',
        'target_ident': 'target',
    }
)

data.head()

,text,label,target
0,I hate women.,hateful,women
1,I hate trans people.,hateful,trans people
2,I hate gay people.,hateful,gay people
3,I hate black people.,hateful,black people
4,I hate disabled people.,hateful,disabled people


It appears as though non of the missing target data have any relevant hate speech associated with them. This can easily be varified by manually looking at the dataset (given its tiny size). But alas, given my horrendous dyslexia i decided to programmatically varify it as a sanity check

In [234]:
target_search_words = [
    'gay',
    'women',
    'disabled',
    'Muslims',
    'black',
    'trans',
    'immigrants'
]

# Get all rows targets with NaNs
data_target_nans = data[data['target'].isna()]

for index, row in data_target_nans.iterrows():
    for word in target_search_words:
        # Split by words and search each one
        if word in row['text'].split(' '):
            print(f'''found target word in: "{row['text']}" row''')

I'm now confident in dropping all rows with target NaN values

In [235]:
data = data.dropna()

Balance the dataset

In [236]:
hateful = data['label'].value_counts()['hateful']
non_hateful = data['label'].value_counts()['non-hateful']

# Because we've got far more hateful texts
number_of_rows_to_drop = hateful - non_hateful

# randomly sample the rows and and drop them
rows_indexes_to_remove = data[data['label'] == 'hateful'].sample(n=number_of_rows_to_drop).index
data = data.drop(rows_indexes_to_remove)

display(
    pd.DataFrame(data['label'].value_counts()),
    pd.DataFrame(data['target'].value_counts())
)

,count
label,
hateful,947
non-hateful,947


,count
target,
gay people,324
women,299
black people,267
disabled people,253
trans people,252
Muslims,252
immigrants,247


### Data Augmentation

In [237]:
# def shuffle_sentences(row):
#     word_list = row['text'].split(' ')
#     shuffled_sentance = ' '.join(sample(word_list, len(word_list)))
#     return shuffled_sentance

# shuffled_sentences = data.apply(shuffle_sentences, axis=1)

# data_copy = data.copy()
# data_copy = data_copy.drop('text', axis=1)
# data_copy.insert(1, 'text', shuffled_sentences)

# data = pd.concat([data, data_copy], ignore_index=True)

# data

Shuffle the dataset

In [238]:
# Shuffles and returns the entire dataset as frac=1
data = data.sample(frac=1)

# Reset index after having both balanced and shuffled
data = data.reset_index(drop=True)
data

,text,label,target
0,Do you not realise how hateful you are being w...,non-hateful,black people
1,"People who send me messages like ""You are just...",non-hateful,gay people
2,They are just a bunch of mongs.,hateful,disabled people
3,Muslims like you are absolutefilth.,hateful,Muslims
4,They work with a lot of disabled people.,non-hateful,disabled people
...,...,...,...
1889,My dog is smarter than any gy person could eve...,hateful,gay people
1890,Just another common shemale.,hateful,trans people
1891,"Obvious hate like ""disabled lives aren't worth...",non-hateful,disabled people
1892,B l a c k people are disgusting.,hateful,black people


In [239]:
max_words_to_use = 1000

# Tokenize the text data (convert them into "sequences")
tokenizer = Tokenizer(num_words=max_words_to_use) # Consider only using the top 1000 words, as those 
tokenizer.fit_on_texts(data['text'])
tokenized_data = tokenizer.texts_to_sequences(data['text'])

# printing using loop for easier viewing
for i in range(5):
    print(f'seq {i}: {tokenized_data[i]}')

seq 0: [53, 4, 17, 177, 61, 205, 4, 2, 113, 111, 50, 14, 1, 178]
seq 1: [1, 78, 162, 21, 163, 7, 4, 2, 25, 5, 164, 3, 21, 31, 55, 165]
seq 2: [37, 2, 25, 5, 454, 19, 485]
seq 3: [26, 7, 4, 2, 433]
seq 4: [37, 310, 73, 5, 117, 19, 16, 1]
